In [20]:
import pandas as pd
import lightgbm as lgb
from date import *
from importlib import reload
import model
reload(model)
from model import *
from metric import get_weights, NWRMSLE_log
import numpy as np

In [3]:
df = load_data_in_date_range('./data/train_processed.csv', '2017-05-01', '2017-08-15', 110000000)

Reading batch from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 5823749, mapping...
Mapped, reducing...
Batch done.
Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [5]:
# log target
df = convert_unit_sales(df)

In [15]:
items.head()

,item_nbr,family,class,perishable,weight
0,96995,GROCERY I,1093,0,1.00
1,99197,GROCERY I,1067,0,1.00
2,103501,CLEANING,3008,0,1.00
3,103520,GROCERY I,1028,0,1.00
4,103665,BREAD/BAKERY,2712,1,1.25


In [17]:
items = pd.read_csv('./data/items_encoded.csv')
stores = pd.read_csv('./data/stores_encoded.csv')

In [7]:
!telegram-send "Data is ready"

In [18]:
folds = [('2017-05-01', '2017-06-15', '2017-06-16', '2017-06-30'),
         ('2017-05-16', '2017-06-30', '2017-07-01', '2017-07-15'),
         ('2017-06-01', '2017-07-15', '2017-07-16', '2017-07-31'),
         ('2017-06-16', '2017-07-31', '2017-08-01', '2017-08-15')]

param = {
    'num_leaves':30,
    'objective':'regression_l2',
    'metric':'l2_root',
    'num_threads':4
        }
results = []

for train_start, train_end, test_start, test_end in folds:
    print("Extracting fold...")
    train, test = extract_train_test(df,  train_start, train_end, test_start, test_end)
    
    print("Preparing train...")
    train = fill_empty_sales(train)
    train, ext_cols = extend_dataset(train, items, stores)
    x_cols = ext_cols + ['onpromotion']
    train_X = train[x_cols]
    train_y = train['unit_sales']
    train_weights = get_weights(train['item_nbr'])
    train_dataset = lgb.Dataset(train_X, label=train_y, weight=train_weights)
    #del train
    
    print("Preparing test...")
    test = fill_empty_sales(test)
    test, _ = extend_dataset(test, items, stores)
    test_X = test[x_cols]
    test_y = test['unit_sales']
    test_weights = get_weights(test['item_nbr'])
    test_dataset = lgb.Dataset(test_X, label=test_y, weight=test_weights, reference=train_dataset)
    #del test
    
    print("Training!")
    bst = lgb.train(param, 
                    train_dataset, 
                    200,
                    valid_sets=[test_dataset], 
                    early_stopping_rounds=10, 
                    verbose_eval=True, 
                    feature_name=x_cols, 
                    categorical_feature=ext_cols)
    
    test_y_pred = bst.predict(test_X)
    error = NWRMSLE_log(test_y_pred, test_y, test_weights)
    print('Validation error: {}'.format(error))
    
    results.append((bst.best_iteration, error))

Extracting fold...
Preparing train...
Preparing test...
Training!


C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\lightgbm\basic.py:1007: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['city', 'class', 'cluster', 'family', 'perishable', 'salary', 'state', 'type', 'weekday', 'weekend']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 1.01825
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 1.003
[3]	valid_0's rmse: 0.99033
[4]	valid_0's rmse: 0.979824
[5]	valid_0's rmse: 0.970866
[6]	valid_0's rmse: 0.963483
[7]	valid_0's rmse: 0.957253
[8]	valid_0's rmse: 0.952025
[9]	valid_0's rmse: 0.947281
[10]	valid_0's rmse: 0.943511
[11]	valid_0's rmse: 0.940287
[12]	valid_0's rmse: 0.937481
[13]	valid_0's rmse: 0.935132
[14]	valid_0's rmse: 0.932986
[15]	valid_0's rmse: 0.930839
[16]	valid_0's rmse: 0.928952
[17]	valid_0's rmse: 0.927365
[18]	valid_0's rmse: 0.925962
[19]	valid_0's rmse: 0.924724
[20]	valid_0's rmse: 0.923602
[21]	valid_0's rmse: 0.92273
[22]	valid_0's rmse: 0.921958
[23]	valid_0's rmse: 0.921207
[24]	valid_0's rmse: 0.920585
[25]	valid_0's rmse: 0.919795
[26]	valid_0's rmse: 0.919339
[27]	valid_0's rmse: 0.918494
[28]	valid_0's rmse: 0.918088
[29]	valid_0's rmse: 0.917666
[30]	valid_0's rmse: 0.917069
[31]	valid_0's rmse: 0.916731
[32]	valid_0's rmse: 0.

KeyboardInterrupt: 

In [19]:
!telegram-send "Validation finished. Results: $results."